# Survival Analysis Lab

Complete the following exercises to solidify your knowledge of survival analysis.

In [55]:
import pandas as pd
import numpy as np
# cf.go_offline()

In [21]:
data = pd.read_csv('../data/attrition.csv')
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   int64 
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [22]:
data.isna().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

## 1. Generate and plot a survival function that shows how employee retention rates vary by gender and employee age.

*Tip: If your lines have gaps in them, you can fill them in by using the `fillna(method=ffill)` and the `fillna(method=bfill)` methods and then taking the average. We have provided you with a revised survival function below that you can use for the exercises in this lab*

In [23]:
def survival(data, group_field, time_field, event_field):
    kmf = KaplanMeierFitter()
    results = []

    for i in data[group_field].unique():
        group = data[data[group_field]==i]
        T = group[time_field]
        E = group[event_field]
        kmf.fit(T, E, label=str(i))
        results.append(kmf.survival_function_)

    survival = pd.concat(results, axis=1)
    front_fill = survival.fillna(method='ffill')
    back_fill = survival.fillna(method='bfill')
    smoothed = (front_fill + back_fill) / 2
    return smoothed

## 2. Compare the plot above with one that plots employee retention rates by gender over the number of years the employee has been working for the company.

In [27]:
rates = survival(data, 'Gender', 'YearsAtCompany','RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Antiquity (Years)', yTitle='Retention Rate',
            title='Retention Rates by Antiquity and Gender')

## 3. Let's look at retention rate by gender from a third perspective - the number of years since the employee's last promotion. Generate and plot a survival curve showing this.

In [29]:
rates = survival(data, 'Gender', 'YearsSinceLastPromotion','RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Years since last promotion', yTitle='Retention Rate',
            title='Retention Rates since last promotion')

## 4. Let's switch to looking at retention rates from another demographic perspective: marital status. Generate and plot survival curves for the different marital statuses by number of years at the company.

In [30]:
rates = survival(data, 'MaritalStatus', 'YearsAtCompany','RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Antiquity (Years)', yTitle='Retention Rate',
            title='Retention Rates by Antiquity and Marital Status')

## 5. Let's also look at the marital status curves by employee age. Generate and plot the survival curves showing retention rates by marital status and age.

In [40]:
rates = survival(data, 'MaritalStatus', 'Age', 'RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Age (Years)', yTitle='Retention Rate',
            title='Retention Rates by Age and Marital Status')

## 6. Now that we have looked at the retention rates by gender and marital status individually, let's look at them together. 

Create a new field in the data set that concatenates marital status and gender, and then generate and plot a survival curve that shows the retention by this new field over the age of the employee.

In [62]:
df = data.copy()
df['MSG'] = df.MaritalStatus + ' ' + df.Gender
df.MSG
rates = survival(df, 'MSG', 'Age', 'RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by Marital Status, Gender and Age')

## 6. Let's find out how job satisfaction affects retention rates. Generate and plot survival curves for each level of job satisfaction by number of years at the company.

In [42]:
rates = survival(data, 'RelationshipSatisfaction', 'YearsAtCompany', 'RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Antiquity(Years)', yTitle='Retention Rate',
            title='Retention Rates by Job Satisfaction')

## 7. Let's investigate whether the department the employee works in has an impact on how long they stay with the company. Generate and plot survival curves showing retention by department and years the employee has worked at the company.

In [44]:
rates = survival(data, 'Department', 'YearsAtCompany', 'RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Antiquity(Years)', yTitle='Retention Rate',
            title='Retention Rates by Department')

## 8. From the previous example, it looks like the sales department has the highest attrition. Let's drill down on this and look at what the survival curves for specific job roles within that department look like.

Filter the data set for just the sales department and then generate and plot survival curves by job role and the number of years at the company.

In [45]:
df = data[data['Department']=='Sales']
rates = survival(df, 'JobRole', 'YearsAtCompany', 'RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Antiquity(Years)', yTitle='Retention Rate',
            title='Retention Rates by Job Role')

## 9. Let examine how compensation affects attrition.

- Use the `pd.qcut` method to bin the HourlyRate field into 5 different pay grade categories (Very Low, Low, Moderate, High, and Very High).
- Generate and plot survival curves showing employee retention by pay grade and age.

In [53]:
df = data.copy()
df['Quantile'] = pd.qcut(data.HourlyRate,q=5)
rates = survival(df, 'Quantile', 'YearsAtCompany', 'RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Antiquity(Years)', yTitle='Retention Rate',
            title='Retention Rates by Job Role')

## 10. Finally, let's take a look at how the demands of the job impact employee attrition.

- Create a new field whose values are 'Overtime' or 'Regular Hours' depending on whether there is a Yes or a No in the OverTime field.
- Create a new field that concatenates that field with the BusinessTravel field.
- Generate and plot survival curves showing employee retention based on these conditions and employee age.

In [61]:
df = data.copy()
df['Over_Time'] = np.where(df['OverTime']=='Yes','OverTime','Regular Hours')
df['OTBT'] = df.Over_Time + ' ' + df.BusinessTravel
df.OTBT
rates = survival(df, 'OTBT', 'Age', 'RelationshipSatisfaction')
rates.iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by Overtime, Business Travel and Age')